In [12]:
%pip install extraPackages/fixedpoint-1.0.1-py3-none-any.whl

Processing ./extraPackages/fixedpoint-1.0.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [13]:
from fixedpoint import FixedPoint
from pynq import allocate
import numpy as np

In [32]:
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/linRegOverlays/directImp_v2.bit')

In [33]:
thetaCalc = overlay.genTheta
thetaCalc?

In [34]:
#Actual numbers

X_float = [-1.1, 0.1, 1.2, 2.3, 3.1, 4.1, 4.8, 5.7]
y_float = [-1.7, 2.4, 5.0, 7.3, 10.9, 12.5, 16.2, 19.7]

In [5]:
#Test numbers to test if IP is doing things or simply printing init

X_float = [-2007.1, 2003.1, 2012.2, 2006.3, 2013.1, 2010.1, 2016.8, 2020.7]
y_float = [-2006.7, 2005.4, 2015.0, 2007.3, 2010.9, 2012.5, 2016.2, 2019.7]

In [35]:
X_fixed = [FixedPoint(x, True, 16, 16) for x in X_float]
y_fixed = [FixedPoint(y, True, 16, 16) for y in y_float]
for x in y_fixed:
    print(str(x))

fffe4ccd
00026666
00050000
00074ccd
000ae666
000c8000
00103333
0013b333


In [7]:
xString = ''
for x in X_fixed:
    xString += str(x)
xBytes = bytes(xString, 'utf-8')

yString = ''
for y in y_fixed:
    yString += str(y)
yBytes = bytes(yString, 'utf-8')


print(xBytes)
print(yBytes)

b'fffee6660000199a0001333300024ccd0003199a0004199a0004cccd0005b333'
b'fffe4ccd000266660005000000074ccd000ae666000c8000001033330013b333'


In [36]:
print(X_fixed[0].bits)
print(type(X_fixed[0].bits))
intBits = int(X_fixed[0].bits)
print(type(intBits))

4294895206
<class 'fixedpoint.fixedpoint.FixedPointBits'>
<class 'int'>


In [37]:
intBits = []
for x in X_fixed:
    intBits.append(int(x.bits))
print(intBits)

for y in y_fixed:
    intBits.append(int(y.bits))
print(intBits)

[4294895206, 6554, 78643, 150733, 203162, 268698, 314573, 373555]
[4294895206, 6554, 78643, 150733, 203162, 268698, 314573, 373555, 4294855885, 157286, 327680, 478413, 714342, 819200, 1061683, 1291059]


In [38]:
for i in range(0, 16, 1):
    print(i*8+16)
    thetaCalc.write(i*8+16, intBits[i])
thetaCalc.write(0x00, 1)

16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136


In [39]:


#X's
print(thetaCalc.read(0x10))
print(thetaCalc.read(0x18))
print(thetaCalc.read(0x20))
print(thetaCalc.read(0x28))
print(thetaCalc.read(0x30))
print(thetaCalc.read(0x38))
print(thetaCalc.read(0x40))
print(thetaCalc.read(0x48))


#Y's
print(thetaCalc.read(0x50))
print(thetaCalc.read(0x58))
print(thetaCalc.read(0x60))
print(thetaCalc.read(0x68))
print(thetaCalc.read(0x70))
print(thetaCalc.read(0x78))
print(thetaCalc.read(0x80))
print(thetaCalc.read(0x88))


#Theta's?
print(thetaCalc.read(0x90))
print(thetaCalc.read(0xa0))

4294895206
6554
78643
150733
203162
268698
314573
373555
4294855885
157286
327680
478413
714342
819200
1061683
1291059
210
18879


In [40]:
out = thetaCalc.read(0xa0)
print(out)
print(out.to_bytes(4, 'big'))
print(type(out.to_bytes(4, 'big')))
print(hex(out))

18879
b'\x00\x00I\xbf'
<class 'bytes'>
0x49bf


In [41]:
# Q16.16
qformat = {'signed': True, 'm': 16, 'n': 16}

# A single instance
x = FixedPoint(0, **qformat)
print(x)
x.from_string(hex(out))
print(x)
float(x)

00000000
000049bf


0.2880706787109375

In [44]:
# Q16.16
qformat = {'signed': True, 'm': 16, 'n': 16}

# A single instance
x = FixedPoint(0, **qformat)
print(x)
x.from_string(hex(thetaCalc.read(0x10)))
print(x)
float(x)

00000000
fffee666


-1.100006103515625

In [27]:
# Q16.16
qformat = {'signed': True, 'm': 16, 'n': 16}

# A single instance
x = FixedPoint(0, **qformat)
print(x)
x.from_string(hex(0x00000200))
print(x)
float(x)

00000000
00000200


0.0078125

In [43]:
# Q16.16
qformat = {'signed': True, 'm': 16, 'n': 16}

# A single instance
x = FixedPoint(0, **qformat)
print(x)
x.from_float(0.003213)
print(x)
float(x)

00000000
000000d3


0.0032196044921875

In [25]:
type(hex(1024))

str

In [133]:
thetaCalc.write(0x00, 1)


In [53]:
print(thetaCalc.read(0x10))
print(thetaCalc.read(16))

4294895206
4294895206


In [13]:
#X's
print(thetaCalc.read(0x10))
print(thetaCalc.read(0x18))
print(thetaCalc.read(0x20))
print(thetaCalc.read(0x28))
print(thetaCalc.read(0x30))
print(thetaCalc.read(0x38))
print(thetaCalc.read(0x40))
print(thetaCalc.read(0x48))


#Y's
print(thetaCalc.read(0x50))
print(thetaCalc.read(0x58))
print(thetaCalc.read(0x60))
print(thetaCalc.read(0x68))
print(thetaCalc.read(0x70))
print(thetaCalc.read(0x78))
print(thetaCalc.read(0x80))
print(thetaCalc.read(0x88))


#Theta's?
print(thetaCalc.read(0x90))
print(thetaCalc.read(0xa0))

1701209702
808464432
825241648
842018864
858796080
875573296
875573296
892350512
1701209702
842018864
892350512
925904944
1630548016
1664102448
808529968
858861616
0
3610303960
